In [1]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [3]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

docs = []
for doc in ["documents/signoff.docx", "documents/commentary.docx"]:
    loader = UnstructuredWordDocumentLoader(doc)
    docs.extend(loader.load())


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [6]:
# Simple RAG chain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [7]:
response = rag_chain.invoke({"input": "How do I open commentary box?"})
response["answer"]


'To open the commentary box, first click on the work package assigned to you. Then, select the unit of work you want to work on, which will take you to the balance sheet page and highlight the row in yellow. Click the highlighted row to open a popup containing the commentary form.'

In [8]:
# Creating conversation RAG chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [10]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "How do I open commentary box?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
print(ai_msg_1["answer"])
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)


To open the commentary box, click on the unit of work you want to work on, which will direct you to the balance sheet page. Then, click the highlighted row that appears in yellow to open a new popup containing the commentary form.


In [11]:
second_question = "How to go about filling it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])


To fill the commentary form, enter your commentary message (between 10 – 2500 characters), select a valid date (not a future date), and choose the commentary type. Once all fields are filled correctly, click the Add button at the bottom right of the popup. If any fields are missing or invalid, the Add button will be disabled, so check for red-highlighted boxes.


In [12]:
# Store chat history using sessions

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


In [13]:
conversational_rag_chain.invoke(
    {"input": "I am unable to provide signoff."},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


"To provide signoff, ensure you have selected the correct workpackage from the left pane and that it is assigned to you. If it's assigned to someone else, contact them to unassign themselves or reach out to the Central Team for assistance. Once you have access, follow the steps to complete the signoff process."

In [14]:
conversational_rag_chain.invoke(
    {"input": "I am able to reach the signoff popup, but even though all fields are filled the button is still disabled."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]


'If the signoff button is still disabled despite filling all fields, double-check that the caveat commentary meets the required character count of 10 – 3500 characters. Additionally, ensure that all necessary questions in the popup have been answered with either "yes" or "no." If everything seems correct and the button remains disabled, there may be a technical issue, and you might need to contact support.'

In [15]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")


User: I am unable to provide signoff.

AI: To provide signoff, ensure you have selected the correct workpackage from the left pane and that it is assigned to you. If it's assigned to someone else, contact them to unassign themselves or reach out to the Central Team for assistance. Once you have access, follow the steps to complete the signoff process.

User: I am able to reach the signoff popup, but even though all fields are filled the button is still disabled.

AI: If the signoff button is still disabled despite filling all fields, double-check that the caveat commentary meets the required character count of 10 – 3500 characters. Additionally, ensure that all necessary questions in the popup have been answered with either "yes" or "no." If everything seems correct and the button remains disabled, there may be a technical issue, and you might need to contact support.

